In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import shutil
import os

In [2]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
df = pd.read_csv('/kaggle/input/allcsv/all.csv')
df

,img_path,text
0,cropped/Fuente3_parte_amistoso_4_23_crop0.jpg,10/8/2004
1,cropped/Fuente3_parte_amistoso_4_23_crop1.jpg,19:23
2,cropped/Fuente3_parte_amistoso_4_23_crop2.jpg,Espana
3,cropped/Fuente3_parte_amistoso_4_23_crop4.jpg,X
4,cropped/Fuente3_parte_amistoso_4_23_crop5.jpg,Tapioles
...,...,...
144992,cropped/Fuente5_parte_amistoso_9_37_crop71.jpg,25/2/2026
144993,cropped/Fuente5_parte_amistoso_9_37_crop72.jpg,ECONOMETRO RAYADO
144994,cropped/Fuente5_parte_amistoso_9_37_crop73.jpg,X
144995,cropped/Fuente5_parte_amistoso_9_37_crop75.jpg,X


In [3]:
# Extract 'Fuente' from 'img_path'
""""
df['Fuente'] = df['img_path'].str.extract(r'(Fuente\d)')

df_fin = df.groupby(['Fuente', 'text']).head(6)

df_fin.reset_index(drop=True, inplace=True)

df_fin = df_fin.drop('Fuente', axis=1)

"""
df = df[~df['img_path'].str.contains('Fuente3|Fuente5')]
df

,img_path,text
148,cropped/Fuente0_parte_amistoso_8_5_crop0.jpg,28/2/2000
149,cropped/Fuente0_parte_amistoso_8_5_crop1.jpg,17:23
150,cropped/Fuente0_parte_amistoso_8_5_crop2.jpg,Espana
151,cropped/Fuente0_parte_amistoso_8_5_crop4.jpg,X
152,cropped/Fuente0_parte_amistoso_8_5_crop5.jpg,Benabarre
...,...,...
144919,cropped/Fuente4_parte_amistoso_4_8_crop70.jpg,a1
144920,cropped/Fuente4_parte_amistoso_4_8_crop71.jpg,5/12/2021
144921,cropped/Fuente4_parte_amistoso_4_8_crop72.jpg,puerta delantera estropeado
144922,cropped/Fuente4_parte_amistoso_4_8_crop74.jpg,x


In [4]:
df['Fuente'] = df['img_path'].str.extract(r'(Fuente\d)')

df_fin = df.groupby(['Fuente', 'text']).head(10)

df_fin.reset_index(drop=True, inplace=True)

df_fin = df_fin.drop('Fuente', axis=1)
df_fin

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,img_path,text
0,cropped/Fuente0_parte_amistoso_8_5_crop0.jpg,28/2/2000
1,cropped/Fuente0_parte_amistoso_8_5_crop1.jpg,17:23
2,cropped/Fuente0_parte_amistoso_8_5_crop2.jpg,Espana
3,cropped/Fuente0_parte_amistoso_8_5_crop4.jpg,X
4,cropped/Fuente0_parte_amistoso_8_5_crop5.jpg,Benabarre
...,...,...
24545,cropped/Fuente6_parte_amistoso_4_3_crop67.jpg,Italia
24546,cropped/Fuente6_parte_amistoso_4_3_crop68.jpg,Benitez.antonio@hotmail.com
24547,cropped/Fuente6_parte_amistoso_4_3_crop69.jpg,56280981h
24548,cropped/Fuente6_parte_amistoso_4_3_crop71.jpg,11/9/2039


In [5]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
train_df = df_fin.copy()
train_df['img_path']= '/kaggle/input/cropped/'+train_df['img_path']

test_df = pd.read_csv('/kaggle/input/allcsv/all.csv').iloc[17000:17005]
test_df['img_path']= '/kaggle/input/cropped/'+test_df['img_path']

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['img_path'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [6]:
train_df = train_df.reset_index()
test_df = test_df.reset_index()

In [7]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained('/kaggle/input/tr-ocr/trocr-small')
train_dataset = IAMDataset(root_dir='.',
                           df=train_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir='.',
                           df=test_df,
                           processor=processor)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [8]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 24550
Number of validation examples: 5


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # Choose the second GPU or CPU


In [10]:
encoding = train_dataset[0]
for k,v in encoding.items():
  print(k, v.shape)

pixel_values torch.Size([3, 384, 384])
labels torch.Size([128])


In [11]:
image = Image.open(train_df['img_path'][0]).convert("RGB")
image

In [12]:
labels = encoding['labels']
labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.decode(labels, skip_special_tokens=True)
print(label_str)

28/2/2000


In [13]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=24, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=24)

In [14]:
from transformers import VisionEncoderDecoderModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VisionEncoderDecoderModel.from_pretrained("/kaggle/input/tr-ocr/trocr-small")
model.to(device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at /kaggle/input/tr-ocr/trocr-small and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


VisionEncoderDecoderModel(
  (encoder): DeiTModel(
    (embeddings): DeiTEmbeddings(
      (patch_embeddings): DeiTPatchEmbeddings(
        (projection): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DeiTEncoder(
      (layer): ModuleList(
        (0): DeiTLayer(
          (attention): DeiTAttention(
            (attention): DeiTSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): DeiTSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): DeiTIntermediate(
            (dense): Linear(in_featu

In [15]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [16]:
!pip install jiwer jiwer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.4 MB/s eta 0:00:00
  Attempting uninstall: rapidfuzz
    Found existing installation: rapidfuzz 2.15.0
    Uninstalling rapidfuzz-2.15.0:
      Successfully uninstalled rapidfuzz-2.15.0


In [17]:
from datasets import load_metric

cer_metric = load_metric("cer")

def compute_cer(pred_ids, label_ids):
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return cer

In [18]:
from transformers import AdamW
from tqdm.notebook import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):  # loop over the dataset multiple times
   # train
   model.train()
   train_loss = 0.0
   for batch in tqdm(train_dataloader):
      # get the inputs
      for k,v in batch.items():
        batch[k] = v.to(device)

      # forward + backward + optimize
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      train_loss += loss.item()

   print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))
    
   # evaluate
   model.eval()
   valid_cer = 0.0
   with torch.no_grad():
     for batch in tqdm(eval_dataloader):
       # run batch generation
       outputs = model.generate(batch["pixel_values"].to(device))
       # compute metrics
       cer = compute_cer(pred_ids=outputs, label_ids=batch["labels"])
       valid_cer += cer 

   print("Validation CER:", valid_cer / len(eval_dataloader))

model.save_pretrained(".")

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/1023 [00:00<?, ?it/s]

Loss after epoch 0: 1.28177726111454


  0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1202: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"
/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1292: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


...

Validation CER: 0.0


  0%|          | 0/1023 [00:00<?, ?it/s]

Loss after epoch 2: 0.09722159191727536


  0%|          | 0/1 [00:00<?, ?it/s]

Validation CER: 0.045454545454545456
